# Training a Word embedding model

Author: 
* Luis

Status:
* DONE

Goal:
* We will train a couple of word embedding models to generate numerical features from our text variables. Specifically we will train a fasttext model using the gensim library.



In [1]:
%cd ..

/home/luis/ds4a/notebooks


## Loading and cleaning data

In [2]:
from src.loading import load_dataset
from src.cleaning import build_df_from_RA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import unicodedata


df = build_df_from_RA(load_dataset("dataset"))
print(f"We have a total of {df.shape[0]} reviews!")

We have a total of 51655 reviews!


In [3]:
import string
import re

def normalize_text(text):
    """
    Strip accents and lower text string
    :param text: (str) text to be cleaned
    :return: (str) cleaned text
    """
    text = strip_accents(text)
    text = text.lower().strip()
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text


def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def remove_numbers(text):
    return re.sub(r'\b[0-9]+\b', '', text)


texts = df.apply(lambda row: f"{remove_numbers(normalize_text(row['title']))} {remove_numbers(normalize_text(row['description']))}", axis=1)

In [4]:
texts.head().values

array(['dor na coluna forte prezados boa noitegostaria que enviasse um tecnico para uma vistoria e possivel troca da camaestou sentindo muitas dores na coluna e parte da cervical fiz exames e deu desvio na coluna o medico alegou que pode ser a camaquando deito sinto as molas tendo um atrito e parece que elas vao entrar no meu corpo as vezes da medo e no inicio nao era assim como comprei a cama e ainda esta na garantia preciso que isso seja resolvido da melhor forma e para que nao piore essas minhas dores estou tomando remedio para dormir com dores fortes fico no aguardo de um breve retornovania melo',
       'simples troca de travesseiros sem retorno comprei uma cama na loja king star do analia franco juntamente com  travesseiros capa de colchao e saia para box a cama chegou ok os travesseiros porem sao extremamente altos e nao notamos no momento da compra  dias apos a compra fui ate a loja para trocalos e me informaram que o procedimento exigia que mandassem um email para a supervisao

In [5]:
def tokenize(data, sep=None):
    if sep is not None:
        return data.split(sep)
    return data.split()

# Create token vectors
tokens_sq = [tokenize(i) for i in texts.values]
len(tokens_sq)

51655

## Training

In [7]:
from pprint import pprint as print
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

seed = 2020
epochs = 100
ft_params = {"sg": 0,  # Training algorithm: skip-gram if sg=1, otherwise CBOW.
             "hs": 0,  #  If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
             "size": 100,  # Dimensionality of the word vectors.
             "alpha": 0.025,  # The initial learning rate.
             "window": 5, # The maximum distance between the current and predicted word within a sentence.
             "min_count": 5,  # The model ignores all words with total frequency lower than this.
             "max_vocab_size": None, 
             "word_ngrams": 1, 
             "sample": 0.001,  # the threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
             "seed": seed,
             "workers": 3, # Use these many worker threads to train the model (=faster training with multicore machines).
             "min_alpha": 0.0001, 
             "negative": 5,  #  If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
             "ns_exponent": 0.75, 
             "cbow_mean": 1,  # If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
             "iter": 5, 
             "min_n": 3, # Minimum length of char n-grams to be used for training word representations.
             "max_n": 6, #  Max length of char ngrams to be used for training word representations. Set max_n to be lesser than min_n to avoid char ngrams being used.
             "sorted_vocab": 1}


model = FT_gensim(**ft_params)

# build the vocabulary
model.build_vocab(sentences=tokens_sq)

In [8]:
# train the model
model.train(
    sentences=tokens_sq, epochs=epochs,
    total_examples=len(tokens_sq)
)

In [10]:
# saving a model trained via Gensim's fastText implementation
import tempfile
import os

with tempfile.NamedTemporaryFile(prefix='model_ft', delete=False) as tmp:
    model.save(tmp.name, separately=[])

# loaded_model = FT_gensim.load(tmp.name)
# print(loaded_model)

# os.unlink(tmp.name)

## Word similarity analysis

In [13]:
loaded_model = FT_gensim.load(str(Path().cwd() / "fasttext.model"))
print(loaded_model)

In [17]:
print(model.wv.most_similar("fabrispuma"))

[('spuma', 0.724884033203125),
 ('kingstar', 0.6920073628425598),
 ('inovar', 0.6686462163925171),
 ('lukspuma', 0.6159020066261292),
 ('marabraz', 0.5994600653648376),
 ('lojamarabraz', 0.5932378768920898),
 ('marabraza', 0.5812759399414062),
 ('viggore', 0.5809564590454102),
 ('inovakasa', 0.5581468939781189),
 ('kingsize', 0.5520431399345398)]


In [19]:
print(model.wv.most_similar("mobly"))

[('marabraz', 0.8294726014137268),
 ('etna', 0.8019948601722717),
 ('madeiramadeira', 0.7776471376419067),
 ('marabraza', 0.7694597244262695),
 ('marabrazcom', 0.7617514729499817),
 ('tokstok', 0.7577101588249207),
 ('marabrazno', 0.7471649646759033),
 ('magalu', 0.7353067994117737),
 ('americanascom', 0.7303666472434998),
 ('aetna', 0.7297446727752686)]


In [16]:
print(model.wv.most_similar("palhaco"))

[('palhaca', 0.8603098392486572),
 ('idiota', 0.856789231300354),
 ('palhacos', 0.827946662902832),
 ('trouxa', 0.7834070920944214),
 ('palha', 0.770859956741333),
 ('bobo', 0.7582768797874451),
 ('idiotas', 0.7374000549316406),
 ('besta', 0.6986215114593506),
 ('boba', 0.6956204175949097),
 ('trouxas', 0.6653217077255249)]


In [20]:
print(model.wv.most_similar("quebrado"))

[('quebradoo', 0.926303505897522),
 ('danificado', 0.8744300603866577),
 ('trincado', 0.8529925346374512),
 ('amassado', 0.8481569886207581),
 ('arranhado', 0.8447990417480469),
 ('emperrado', 0.8432565927505493),
 ('rasgado', 0.8348040580749512),
 ('avariado', 0.8346461057662964),
 ('rachado', 0.8194884657859802),
 ('lascado', 0.8117988705635071)]


### Remarks

The results look very solid. When we search for company names, it returns other company names as most similar words. When we search for words that have multiple meaning, they return similar words that actually have similar meaning in this context

## Model fit analysis

In [22]:
token_seq_to_str = lambda x: " ".join(x)
ex = tokens_sq[0]
token_seq_to_str(ex)

'dor na coluna forte prezados boa noitegostaria que enviasse um tecnico para uma vistoria e possivel troca da camaestou sentindo muitas dores na coluna e parte da cervical fiz exames e deu desvio na coluna o medico alegou que pode ser a camaquando deito sinto as molas tendo um atrito e parece que elas vao entrar no meu corpo as vezes da medo e no inicio nao era assim como comprei a cama e ainda esta na garantia preciso que isso seja resolvido da melhor forma e para que nao piore essas minhas dores estou tomando remedio para dormir com dores fortes fico no aguardo de um breve retornovania melo'

In [99]:
def get_review_vec(review_seq):
    try:
        return np.array([model.wv[w] for w in review_seq]).mean(axis=0)
    except:
        return None 

texts_vecs = [get_review_vec(seq) for seq in tokens_sq]


/home/luis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/luis/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [102]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

idx_vec = 30057
distances = []
cos_sim = []

for idx,vec in enumerate(texts_vecs):
    if idx != idx_vec and vec is not None:
        try:
            cos_sim.append(cosine_similarity(texts_vecs[idx_vec].reshape(1,-1), vec.reshape(1,-1))[0][0])
        except:
            cos_sim.append(None)
        try:
            distances.append(euclidean_distances(texts_vecs[idx_vec].reshape(1,-1), vec.reshape(1,-1))[0][0])
        except:
            distances.append(None)
        
    else:
        distances.append(None)
        cos_sim.append(None)



In [103]:
simil_df = pd.DataFrame(np.array([distances, cos_sim]).transpose(), columns=["dist", "cosine"]).dropna()
simil_df.head()

,dist,cosine
0,19.7732,-0.0700241
1,18.3038,0.171999
2,16.2397,0.416134
3,15.8188,0.46035
4,15.8395,0.464085


In [104]:
print(token_seq_to_str(tokens_sq[idx_vec]))

for idx in simil_df.sort_values(by="dist",ascending=True).head(10).index:    
    print("=====================================")
    print(token_seq_to_str(tokens_sq[idx]))


'produto foi entregue quebrado recebi o produto quebrado'
'====================================='
('produto nao entregue recebi um email da loja informando que o produto '
 'torradeira pedido n foi entregue porem nao foi fiquei em casa o dia ionteiro '
 'e eles informaram que o produto foi entregue')
'====================================='
('pedido cancelado sem solicitacao pessimo atendimento cooktop tramontina '
 'comprado em deveria ser entregue em pedido q90603575 o pedido nao foi '
 'entregue na data e entao foi aberto um chamado para verificar a data de '
 'entrega apos o evento foi informado pelo atendente que o pedido seria '
 'entregue em hoje quando fui verificar o status do pedido me consta que o '
 'pedido foi cancelado em momento algum foi solicitado o cancelamento do '
 'pedido o valor ja foi pago debito favor informar o status do pedido urgente')
'====================================='
('produto nao entregue fui verificar o status do meu pedido e o mesmo esta '
 'como en

3

In [105]:
idx_vec = np.random.randint(0, len(texts_vecs))
distances = []
cos_sim = []

for idx,vec in enumerate(texts_vecs):
    if idx != idx_vec and vec is not None:
        try:
            cos_sim.append(cosine_similarity(texts_vecs[idx_vec].reshape(1,-1), vec.reshape(1,-1))[0][0])
        except:
            cos_sim.append(None)
        try:
            distances.append(euclidean_distances(texts_vecs[idx_vec].reshape(1,-1), vec.reshape(1,-1))[0][0])
        except:
            distances.append(None)
        
    else:
        distances.append(None)
        cos_sim.append(None)
        
simil_df = pd.DataFrame(np.array([distances, cos_sim]).transpose(), columns=["dist", "cosine"]).dropna()
simil_df.head()

print(token_seq_to_str(tokens_sq[idx_vec]))
for idx in simil_df.sort_values(by="dist",ascending=True).head(10).index:    
    print("=====================================")
    print(token_seq_to_str(tokens_sq[idx]))

('cobranca indevida de parcela minima ao realizar uma negociacao dentro da '
 'loja fisica da empresa tokstok de uma cama escorregador que estava em '
 'promocao devido ser peca de mostruario questionei a supervisora renata a '
 'forma de parcelamento da mesma e fui informado que a quantidade maxima seria '
 'em ate vezes sem juros ao questionar que queria em dez vezes sem juros a '
 'mesma passou a informacao que a loja possui um valor minimo de parcela no '
 'valor de ferindo a lei artigo e artigo que diz que nao se pode estipular '
 'valor minimo de parcelo e ao mesmo tempo obrigando o cliente a comprar '
 'outros produtos para conseguir a quantidade de parcelas pretendidas')
'====================================='
('atendimento bom diaem por volta de hrs fui fazer a retirada de uma tvna '
 'unidade de venda nova rua padre pedro pinto1166 produto oriundo da nota '
 'fiscal de numero 00101446tv led fhdlg43lm6300psb bthdmiwifio atendimento foi '
 'normal ate a hora da retirada do prod

## Remarks

Yes, another confirmation of good fit. First case we got a problem of delivery and it was able to find similar delivery related complaints. Next, it does the same for wrong charges complaints.
We will proceed using this model to our further nlp needs.